In [14]:
import re
import json
import pandas as pd
import ollama

from tqdm import tqdm
from transformers import BertTokenizer

ModuleNotFoundError: No module named 'ollama'

In [6]:
%reload_ext autoreload
%autoreload 2
# 這會從同一個資料夾底下的 process_long_text_by_ollama.py 檔案裡面找到對應的函數 import
from process_long_text_by_ollama import process_long_text_by_ollama 

In [3]:
df = pd.read_csv("dat/cnn_dailymail_test.csv")

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
my_model_name = "compressor"
ollama.create(my_model_name, modelfile='''FROM llama3.1:latest
PARAMETER temperature 0.2
PARAMETER num_ctx 2048
SYSTEM """
- You can ONLY remove unimportant words.
- Do not reorder the original words.
- Do not change the original words.
- Do not use abbreviations or emojis.
- Do not add new words or symbols.
- Provide ONLY the result text, without any explanations or additional information.
"""
''')
print("MODEL you have: ")
for model in ollama.list()['models']:
    YesNo = "⭕️" if (my_model_name in model["name"]) else "❌"
    print(f"{YesNo} {model['name']}")

MODEL you have: 
⭕️ compressor:latest
❌ qwen2.5:latest
❌ llama3.1:latest
❌ yi:latest
❌ llama-pro:latest
❌ llama3:latest


In [12]:
SAVE_RESULT_PATH = "./dat/highlighted-cnn_dailymail_test.dat" # 妳要儲存的黨名
SAVE_EVERRY_NUM = 2 # 每跑完幾筆資料就儲存
TEMPORARY_SAVE_STRING = "" # 用於儲存要放入文件的字串
START_INDEX = 6056 # 開始的 index (不會包含這個index，會接下去會跑index+1, index+2,...)
import time
for index, row in tqdm(df.iloc[START_INDEX:].iterrows(), total=len(df)-START_INDEX, desc="highlight articles"):
    if index < START_INDEX: continue # 跳過索引小於 START_INDEX 的資料
    try:
        article_id = row['id']
        article = row['article']
        highlight_article = process_long_text_by_ollama("highlight", article, my_model_name, tokenizer)
        
        TEMPORARY_SAVE_STRING += f"{article_id}, {json.dumps(highlight_article)}\n"
        
        if(index%SAVE_EVERRY_NUM):
            with open(SAVE_RESULT_PATH, "a") as file:
                file.write(TEMPORARY_SAVE_STRING)
            TEMPORARY_SAVE_STRING = ""
    except Exception as e:
        print(f"Error processing row {index}: {e}")
            
print("Processing complete and all highlights have been saved step-by-step in JSON.")

highlight articles:   0%|                              | 0/5434 [00:07<?, ?it/s]


KeyboardInterrupt: 